# Survey_statistics

Survey feature map's mean & std for each layer in model, for epoch in train-process.

In [1]:
import torch
from ignite.utils import manual_seed

from analyze import *

manual_seed(123)

In [2]:
experiment_dir = 

In [ ]:
class Surveyer:
    def __init__(self, experiment_dir):
        self.experiment_dir = experiment_dir
        self.mean = {}
        self.std = {}
        
        
    def record_fmap_mean_and_std(self, model, epoch):
        self.mean[epoch] = {}
        self.std[epoch] = {}
        for n, m in model.named_modules():
            if isinstance(m, torch.nn.BatchNorm2d):
                self.mean[epoch][n] = m.running_mean
                self.std[epoch][n] = m.running_var
        
        
    def survey(self):
        for epoch in range(0, 90 + 1, 5):
            model, exp_config = load_model(self.experiment_dir, epoch)
            self.record_fmap_and_std(model, epoch)

In [4]:
sv = Surveyer('EoQ - ResNet18+ImageNet/21')
sv.survey()

print(sv.mean)
print(sv.std)

experiment configuration: 
{'dataloader': {'dataset': 'ImageNet',
                'eval_loader_params': {'batch_size': 256,
                                       'num_workers': 8,
                                       'pin_memory': True,
                                       'shuffle': False},
                'train_loader_params': {'batch_size': 256,
                                        'num_workers': 8,
                                        'pin_memory': True,
                                        'shuffle': True}},
 'datetime': '20210511',
 'experiment': {'ID': 21, 'title': 'EoQ - ResNet18+ImageNet'},
 'gradient_clipping': None,
 'loss_function': {'type': 'CrossEntropyLoss'},
 'model': {'backbone': 'resnet18',
           'variation': {'config': {'No SilhouetteFlow': {'activation': 8,
                                                          'weight': 8},
                                    'predictors': {'before layer1': {'editor': {'k': 5,
                                

NameError: name 'resnet18_QAT' is not defined